# Robot Programming with Lisp - 3. Object-Oriented Programming and Failure Handling

## Overview

* Structures
* Hash Tables
* Common Lisp Object System (CLOS)
* Generic Programming
* Failure Handling
* Organizational and Links

## Structures

In [ ]:
;; Handling Structs
(defstruct player
    id
    (name "mysterious stranger" :type string)
    (hp 10 :type integer)
    (mp 0 :type integer)
    and-so-on)

In [ ]:
(defvar *player* (make-player :name "Turtle" :and-so-on '123))

*player*

In [ ]:
(player-name *)

In [ ]:
(defvar *player-copy* (copy-player *player*))

In [ ]:
(setf (player-name *player-copy*) "Cat")

*player-copy

In [ ]:
*player*

## Hash Tables

In [ ]:
;; Handling Hash Tables
(defvar *table* (make-hash-table :test 'equal))

*table*

In [ ]:
(setf (gethash "MZH" *table*) "Bibliothekstrasse 3"
      (gethash "TAB" *table*) "Am Fallturm 1")

In [ ]:
(gethash "MZH" *table*)

## Common Lisp Object System (CLOS)

In [ ]:
;; Handling Classes
(defclass shape ()
    ((color :accessor get-shape-color
            :initarg :set-color)
     (center :accessor shape-center
             :initarg :center
             :initform '(0 . 0))))

In [ ]:
(defvar *red-shape* (make-instance 'shape :set-color 'red))

In [ ]:
(describe *red-shape*)

In [ ]:
(get-shape-color *red-shape*)

In [ ]:
;; Inheritance
(defclass circle (shape)
    ((radius :initarg :radius)))

In [ ]:
(defvar *circle*
    (make-instance 'circle :set-color 'green :radius 10))

In [ ]:
(describe *circle*)

In [ ]:
(slot-value *circle* 'radius)

### Lisp class vs. Java class

Lisp classes have / support:
* attributes
* getter-setter methods
* multiple inheritance

Lisp classes don’t have:
* attribute access specifications (managed with package namespaces)
* methods

## Generic Programming

### Function Overloading: Generic Programming

In [ ]:
;; Defining Generic Functions
(defgeneric area (x)
    (:documentation "Calculates area of object of type SHAPE."))

In [ ]:
(area 1)  ;; no applicable method...

In [ ]:
(defmethod area (x)
    (error "AREA is only applicable to SHAPE instances"))

In [ ]:
(defmethod area ((obj shape))
    (error "We need more information about OBJ to know its area"))

In [ ]:
(defmethod area ((obj circle))
    (* pi (expt (slot-value obj 'radius) 2)))

In [ ]:
(area 1)

In [ ]:
(area *red-shape*)

In [ ]:
(area *circle*)

In [ ]:
;; Method combinations: :before, :after, :around
(defmethod area :before (obj)
    (format t "Before area. "))

In [ ]:
(area *circle*)

In [ ]:
(defmethod area :around ((obj shape))
    (format t "Taking over shape area. "))

In [ ]:
(area *red-shape*)

In [ ]:
(defmethod area :around ((obj shape))
    (format t "Taking over shape area. ")
    (call-next-method))

In [ ]:
(area *red-shape*)

In [ ]:
(defmethod area :around ((obj shape))
    (round (call-next-method)))

In [ ]:
(area *circle*)

In [ ]:
;; Custom :method-combination
(defgeneric awesome-function (x)
    (:method-combination +))

In [ ]:
(defmethod awesome-function + ((x number))
    x)

In [ ]:
(awesome-function 2)

In [ ]:
(typep 2 'number)

In [ ]:
(typep 2 'integer)

In [ ]:
(defmethod awesome-function + ((x integer))
    x)

In [ ]:
(awesome-function 2)

### OOP in Lisp - Summary
OOP:
* Everything is an object.
* Objects interact with each other.
* Methods 'belong' to objects.

Functional programming:
* Everything is a function.
* Functions interact with each other.
* Objects 'belong' to (generic) functions.

OOP principles in Lisp:
* inheritance (`defclass`)
* encapsulation (closures)
* subtyping polymorphism (`defclass`)
* parametric polymorphism (generic functions)

## Failure Handling

### Invoking Conditions

In [ ]:
;; define-condition, error
(error "oops, something went wrong...")

In [ ]:
(define-condition input-not-a-number (simple-error)
    ((actual-input :initarg :actual-input
                   :reader actual-input
                   :initform nil))
    (:report (lambda (condition stream)
                     (format stream "~a is not a number!"
                             (actual-input condition)))))

In [ ]:
(let ((input (read)))
     (if (numberp input)
         input
         (error (make-condition 'input-not-a-number
                                :actual-input input))))

### Catching Conditions


In [ ]:
;; handler-case
(defparameter *result* nil)
(let ((x (random 3)))
     (setf *result* (/ 123 x))
     (format t "new result is: ~a~%" *result*)
     (setf *result* 0)
     (format t "cleaned up: ~a~%" *result*))

In [ ]:
(defparameter *result* nil)
(let ((x (random 3)))
     (handler-case
         (progn (setf *result* (/ 123 x))
             (format t "new result is: ~a~%" *result*)
             (setf *result* 0)
             (format t "cleaned up: ~a~%" *result*))
         (division-by-zero (error)
                           (format t "~a~%" error)))
     (format t "Final result: ~a~%" *result*))

In [ ]:
;; unwind-protect
(defparameter *result* nil)
(let ((x (random 3)))
     (handler-case
         (unwind-protect
          (progn
              (setf *result* (/ 123 x))
              (format t "new result is: ~a~%" *result*))
          (setf *result* 0)
          (format t "cleaned up: ~a~%" *result*))
         (division-by-zero (error)
                           (format t "~a~%" error)))
     (format t "final result: ~a~%" *result*))

## Assignment

`git pull` for assignment 3! Treasure hunting in an object-oriented world.

Due: 09.11., Wed, 23:59 CEST

Points: 10

Next Class: 10.11. 14:15 CEST

## Links

Cool article by Paul Graham on programming languages:
* http://www.paulgraham.com/avg.html

"Practical Common Lisp" failure handling chapter:
* http://www.gigamonkeys.com/book/beyond-exception-handling-conditions-and-restarts.html